# COMPOSITE TSFRESH

### Create dataframe

In [1]:
import numpy as np
from glob import glob
import pandas as pd
import csv
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor

files = glob('data/*.csv')
totals = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']

df_list = list()
target = list()
n = 200 # number of files.

def get_half_class_df(file_quantity, target_value):
    count = 0
    for i, file in enumerate(files):
        df = pd.read_csv(file)

        if df.iloc[:,-1].unique()[0] == target_value:
            count += 1
            df['id'] = i
            target.append(df.TOTAL_QUALITY.unique()[0])
            df = df.drop(axis=1, columns=totals)   

            consta = [col for col in df if col.endswith(('K1', 'K2', 'K3'))]
            series = [col for col in df if col.endswith(('Time', 'id', 'Flow rate', 'Pressure'))]  

            distributor = MultiprocessingDistributor(n_workers=8, disable_progressbar=True, progressbar_title="Feature Extraction")
            extracted_features = extract_features(df[series], column_id='id', column_sort='Time', distributor=distributor)

            for j in consta:
                extracted_features[j] = df[j].unique()[0] 

            df_list.append(extracted_features)
        if file_quantity == count:
            break

# Get half of class 1
get_half_class_df(n/2, 1)
get_half_class_df(n/2, 0)

df = pd.concat(df_list)
df.head()        

variable  Flow rate__abs_energy  Flow rate__absolute_sum_of_changes  \
id                                                                    
0                  1.031236e-07                            0.000062   
2                  9.353593e-08                            0.000079   
5                  1.097562e-07                            0.000044   
11                 9.292072e-08                            0.000063   
12                 9.332236e-08                            0.000057   

variable  Flow rate__agg_autocorrelation__f_agg_"mean"__maxlag_40  \
id                                                                  
0                                                       0.0         
2                                                       0.0         
5                                                       0.0         
11                                                      0.0         
12                                                      0.0         

variable  Flow rate__agg_autocorrelation__f_agg_"median"__maxlag_40  \
id                                                                    
0                                                       0.0           
2                                                       0.0           
5                                                       0.0           
11                                                      0.0           
12                                                      0.0           

variable  Flow rate__agg_autocorrelation__f_agg_"var"__maxlag_40  \
id                                                                 
0                                                       0.0        
2                                                       0.0        
5                                                       0.0        
11                                                      0.0        
12                                                      0.0        

variable  Flow rate__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"intercept"  \
id                                                                                   
0                                                  0.000012                          
2                                                  0.000013                          
5                                                  0.000014                          
11                                                 0.000014                          
12                                                 0.000014                          

variable  Flow rate__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"  \
id                                                                                
0                                                 -0.243467                       
2                                                 -0.410989                       
5                                                 -0.491959                       
11                                                -0.566665                       
12                                                -0.517404                       

variable  Flow rate__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"  \
id                                                                               
0                                             -4.506888e-08                      
2                                             -7.622757e-08                      
5                                             -1.091336e-07                      
11                                            -1.146758e-07                      
12                                            -1.066454e-07                      

variable  Flow rate__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"  \
id                                                                                
0                                              1.958974e-08                       
2                                 

In [2]:
df['target'] = target
df.to_csv('tmp/extracted_data.csv')

### Read data from file -- 'data/extracted_data.csv'

In [3]:
# code
import pandas as pd

df = pd.read_csv('tmp/extracted_data.csv')
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [4]:
y.value_counts()

1    100
0    100
Name: target, dtype: int64